In [1]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import *

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from glob import glob
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import *

import multiprocessing
from multiprocessing.dummy import Pool

%matplotlib inline

Using TensorFlow backend.


In [2]:
width = 299
height = width

fnames = glob('*_crop/*.png')
np.random.shuffle(fnames)
class0 = len(glob('normal_crop/*.png'))
class1 = len(glob('abnormal_crop/*.png'))

n = len(fnames)

X = np.zeros((n, height, width, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

def f(index):
    return index, cv2.imread(fnames[index])

with Pool(multiprocessing.cpu_count()) as pool:
    with tqdm(pool.imap_unordered(f, range(n)), total=n) as pbar:
        for i, img in pbar:
            X[i] = img
            if 'abnormal' in fnames[i]:
                y[i] = 1
            else:
                y[i] = 0

100%|██████████| 2287/2287 [00:00<00:00, 3341.34it/s]


In [3]:
n_train = int(n*0.8)
X_train = X[:n_train]
y_train = y[:n_train]
X_valid = X[n_train:]
y_valid = y[n_train:]

In [4]:
class RocAucMetricCallback(Callback):
    def __init__(self):
        super(RocAucMetricCallback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        logs['roc_auc_val']=float('-inf')
        y_pred = model.predict(X_valid, batch_size=4)
        auc = roc_auc_score(y_valid, y_pred)
        acc = accuracy_score(y_valid[:,0], y_pred[:,0] > 0.5)
        print(f'ACC: {acc:.4f} AUC: {auc:.4f}')

In [5]:
input_tensor = Input((height, width, 3))

x = Lambda(lambda x:(x-127.0)/127.0)(input_tensor)

for i in range(6):
    x = Conv2D(32*2**i, 3, activation='relu', padding='same')(x)
    x = Conv2D(32*2**i, 3, activation='relu', padding='same')(x)
    x = MaxPool2D(2)(x)

x = GlobalAvgPool2D()(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

In [6]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [7]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[RocAucMetricCallback()])

Epoch 1/10
1829/1829 [==============================] - 15s 8ms/step - loss: 0.7079 - acc: 0.5631
ACC: 0.6114 AUC: 0.8192
Epoch 2/10
1829/1829 [==============================] - 11s 6ms/step - loss: 0.6503 - acc: 0.5992
ACC: 0.7707 AUC: 0.8621
Epoch 3/10
1829/1829 [==============================] - 11s 6ms/step - loss: 0.4501 - acc: 0.8196
ACC: 0.8275 AUC: 0.8294
Epoch 4/10
1829/1829 [==============================] - 11s 6ms/step - loss: 0.3912 - acc: 0.8518
ACC: 0.8341 AUC: 0.8781
Epoch 5/10
1829/1829 [==============================] - 11s 6ms/step - loss: 0.3581 - acc: 0.8639
ACC: 0.8624 AUC: 0.9212
Epoch 6/10
1829/1829 [==============================] - 11s 6ms/step - loss: 0.3184 - acc: 0.8693
ACC: 0.8515 AUC: 0.9154
Epoch 7/10
1829/1829 [==============================] - 11s 6ms/step - loss: 0.3313 - acc: 0.8660
ACC: 0.8581 AUC: 0.9344
Epoch 8/10
1829/1829 [==============================] - 11s 6ms/step - loss: 0.3254 - acc: 0.8814
ACC: 0.8122 AUC: 0.8111
Epoch 9/10
1829/1829 [==

In [8]:
model.save('model.h5')